# 程序框架

- 本教程目的在于教授 G4 的基本运行原理，以及如何快速应用自己的科研中
- 刻意放弃一些类（功能）的使用，例如数据存储中的 hist 等等
- 模拟与数据处理分离
    - 为何模拟与数据处理必须分离？
    - G4程序只负责模拟部分（一次运行）
    - 基于 MakeClass 等框架的数据分析（可重复处理）
    - 创建几何体的过程中，多用可视化界面检查

## 运行需要的基本类

- DetectorConstruction: 继承自 G4VUserDetectorConstruction
- PhysicsList: 继承自 G4VModularPhysicsList
- ActionInitialization: 继承自 G4VUserActionInitialization
    - PrimaryGeneratorAction: 继承自 G4VUserPrimaryGeneratorAction
    - RunAction:继承自 G4UserRunAction
    - EventAction:继承自 G4UserEventAction
    - TrackingAction:继承自 G4UserTrackingAction
    - SteppingAction:继承自 G4UserSteppingAction
    - StackingAction:继承自 G4UserStackingAction

其中，**DetectorConstruction**，**PhysicsList**，**ActionInitialization**，**PrimaryGeneratorAction** 四个类不可缺少，其余几个类根据情况选择使用。

使用者必须熟练掌握探测器的几何设置 **DetectorConstruction** 和粒子源的设置 **PrimaryGeneratorAction** 两个类。

## 单位系统

G4可以进行单位换算，实质上是一些全局变量（G4SystemOfUnits.hh）


- 长度：um, mm, cm, m, km
- 能量：eV, keV, MeV
- 质量：mg, g, kg
- 摩尔质量: mole,
- 体积：mm3, cm3, m3, km3
- 时间：ns, s
- 电荷量：eplus

其中，默认单位为：mm, MeV, g, mole, ns, eplus，即如果没有写单位，则为默认单位。

**为了方便代码阅读，我们要求在数字后面指定单位。**

例如:

```cpp
double x = 5.0*m;
double y = 4.0*m;
double z = 3.0*m;
```

对于输出数据，采用默认单位进行输出。如果要进行单位转换，只需要除以相应的单位即可。

例如:

```cpp
G4cout << KineticEnergy/keV << " keV";
G4cout << density/(g/cm3)   << " g/cm3";
```


- Run 是 GEANT4 最大的模拟单元。
- G4MTRunManager::Initialize() 调用后，初始化 G4 内核
- G4MTRunManager::BeamOn() 调用后，开始一次 G4Run

![111](img/RunEventTrackStep.png)


- 一次 BeamOn 调用就是一次 G4Run; 一次 G4Run 过程中，探测器设置和物理过程不可修改。
- BeamOn(1000) : 一次 G4Run 中，发射 1000 个事件（G4Event）
- 一个粒子的产生到该粒子停止追踪，径迹信息由同一个 G4Track 记录
- 在一个步长中，粒子信息该变量由 G4Step 记录



- G4Step 类包括两个“点”：PreStepPoint，PostStepPoint（对应类均为 G4StepPoint）。
- PreStepPoint 容纳物理过程发生前该粒子的状态（动能，电荷量等），
- PostStepPoint 容纳物理过程发生后该粒子的状态。
- G4Step 中保存的是这两“点”的粒子属性改变量（deltaE 等）

![123](img/steppoint.png)

- 步长：由该 step 的物理过程决定。
- 边界：step 不能穿过两个物理体的边界，即边界上必须存在一个 steppoint。此时 step 的 status 为 fGeomBoundary

- G4Track：定义为粒子当前的状态（与 PostStepPoint 容纳的信息一致）。
- 每个 G4Step 调用后，G4Track 信息更新。
- Track ID: ID 编号从 1 开始，每个 track 中也记录其父粒子的 track ID。
- 初级粒子（particlegun 发射的粒子）的父粒子 Track ID 为 0

![1234](img/trackid.png)

**提取物理量的接口：UserAction,  G4VSensitiveDetector**

- G4UserSteppingAction
    - UserSteppingAction(const G4Step*)
- G4UserTrackingAction
    - PreUserTrackingAction(const G4Track *)
    - PostUserTrackingAction(const G4Track *)
- G4UserEventAction
    - BeginOfEventAction()
    - EndOfEventAction()
- G4UserRunAction
    - BeginOfRunAction()
    - EndOfRunAction()


Example : 
- RunAction ：记录并输出程序运行时间、创建文件
- EventAction : 输出进度、SD 得到该事件的所有数据
- TrackingAction : 输出每个 track 的初始信息、遗传标记、杀死次级粒子
- SteppingAction : 获得该 step 的所有数据


In [1]:
!jupyter nbconvert FRAME.ipynb --to html

[NbConvertApp] Converting notebook FRAME.ipynb to html

[NbConvertApp] Writing 567070 bytes to FRAME.html

